In [50]:
import os

import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
data_dir = '/'.join(os.getcwd().split('/')[:-2] + ['Google Диск', 'Сириус', 'library_recoms'])
print(os.listdir(data_dir))

['materials_actions.csv.gz', 'Icon\r', 'materials_subjects_and_education_levels.csv.gz', 'materials_cis.csv.gz', 'lesson_teacher.csv.gz', 'competences.csv', 'content_type.csv', 'materials.csv.gz']


In [4]:
%%time
df = pd.read_csv(os.path.join(data_dir, 'materials_actions.csv.gz'), sep = ';', compression = 'gzip')

<string>:2: DtypeWarning: Columns (4,5,7,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 4s, sys: 3.92 s, total: 1min 7s
Wall time: 1min 8s


In [5]:
df.head(2)

,material_type,material_id,content_type,logical_type,book_type,game_app_category,game_app_version_id,game_app_version,user_id,profile_id,profile_type,school_id,publisher_id,time_to_study,average_rating,action_type,action_info,action_start,action_end,action_user_id,action_profile_id,action_profile_type,action_school_id
0,Book,24853,NaN,NaN,ХудЛит,NaN,NaN,NaN,13057293.0,13340607.0,PublisherProfile,NaN,383.0,NaN,0.0,загрузка в библиотеку,NaN,2007-06-24 12:26:10,2007-06-24 12:26:10,13057293.0,13340607.0,PublisherProfile,NaN
1,Book,24724,NaN,NaN,ХудЛит,NaN,NaN,NaN,13057293.0,13340607.0,PublisherProfile,NaN,383.0,NaN,0.0,загрузка в библиотеку,NaN,2007-09-25 21:40:27,2007-09-25 21:40:27,13057293.0,13340607.0,PublisherProfile,NaN


## Описание данных materials_actions.
В данном датасете содержится информация о действиях пользователей с материалами.    
В процессе первичного анализа нам нужно ответить на вопросы:   
1. Какие типы действий у нас есть? разобрались, см ниже
2. Какие типы материалов у нас есть? разобрались, см ниже
3. Какие типы контента? разобрались, см ниже
4. Что будет идентификатором материала? это пара полей material_type + material_id
5. Что будет идентификатором пользователя? action_user_id
6. Какие типы пользователя? разобрались, см ниже
7. Какую дату нам брать для дальнейшего анализа?
### Какие типы действий нам будут полезны для формирования рекомендаций:
1. добавление в избранное
2. удаление - фильтруем материалы, убираем из рекомендаций удаленные
3. использование в сценарии урока
4. оценка
### Гипотезы:
1. Возможно, существуют пользователи, у которых есть любимый автор. Надо это проверить.
2. Как влияет рейтинг на популярность материала? Важна не только средняя оценка но и количество проставленных оценок.

In [6]:
df.shape

(10343167, 23)

In [9]:
df.content_type.value_counts()

Изображение    2206904
Видео           592861
Тест            505336
Текст           315415
Аудио            82618
Лаборатория      19589
Таблица            767
Файл                30
Name: content_type, dtype: int64

In [11]:
df.content_type.isnull().sum()

6619647

In [14]:
df.material_type.value_counts()

LessonTemplate       4257925
AtomicObject         3723520
GameApp              1345359
Book                  599085
TestSpecification     297291
ComposedDocument      119898
DigitalDocument           89
Name: material_type, dtype: int64

In [15]:
df.material_type.isnull().sum()

0

In [16]:
df.isnull().sum()

material_type                0
material_id                  0
content_type           6619647
logical_type           7297954
book_type              9744082
game_app_category      9013455
game_app_version_id    8997808
game_app_version       8997808
user_id                   9456
profile_id                9456
profile_type              9456
school_id              1069759
publisher_id           9313622
time_to_study          7248910
average_rating            3730
action_type                  0
action_info            8455320
action_start            433742
action_end              433742
action_user_id             258
action_profile_id        14413
action_profile_type      14416
action_school_id        609736
dtype: int64

In [17]:
df.action_type.value_counts()

просмотр                          4366526
запуск                            1921755
оценка                             862356
добавление в избранное             656239
одобрено модерацией                486097
загрузка в библиотеку              435801
обновлён                           435801
удаление                           435801
копирование                        366647
отправка на модерацию              315259
использование в сценарии урока      56375
использование в ЭУП                  4510
Name: action_type, dtype: int64

In [18]:
df.head(2)

,material_type,material_id,content_type,logical_type,book_type,game_app_category,game_app_version_id,game_app_version,user_id,profile_id,profile_type,school_id,publisher_id,time_to_study,average_rating,action_type,action_info,action_start,action_end,action_user_id,action_profile_id,action_profile_type,action_school_id
0,Book,24853,NaN,NaN,ХудЛит,NaN,NaN,NaN,13057293.0,13340607.0,PublisherProfile,NaN,383.0,NaN,0.0,загрузка в библиотеку,NaN,2007-06-24 12:26:10,2007-06-24 12:26:10,13057293.0,13340607.0,PublisherProfile,NaN
1,Book,24724,NaN,NaN,ХудЛит,NaN,NaN,NaN,13057293.0,13340607.0,PublisherProfile,NaN,383.0,NaN,0.0,загрузка в библиотеку,NaN,2007-09-25 21:40:27,2007-09-25 21:40:27,13057293.0,13340607.0,PublisherProfile,NaN


In [19]:
pd.crosstab(df.material_id, df.material_type)

material_type,AtomicObject,Book,ComposedDocument,DigitalDocument,GameApp,LessonTemplate,TestSpecification
material_id,,,,,,,
1,0,668,0,0,0,0,0
2,0,1753,0,24,0,0,0
3,0,3044,0,0,0,0,0
4,0,1371,0,0,0,0,0
5,0,0,0,15,0,0,0
6,0,0,0,15,0,0,0
7,0,0,0,20,0,0,0
8,0,0,0,15,0,0,0
15,0,0,0,0,153,0,0


In [20]:
df[df.material_type == 'AtomicObject'].content_type.value_counts()

Изображение    2206904
Видео           592861
Тест            505336
Текст           315415
Аудио            82618
Лаборатория      19589
Таблица            767
Файл                30
Name: content_type, dtype: int64

In [21]:
df[df.material_type != 'AtomicObject'].content_type.value_counts()

Series([], Name: content_type, dtype: int64)

In [22]:
df[df.material_id == 2]

,material_type,material_id,content_type,logical_type,book_type,game_app_category,game_app_version_id,game_app_version,user_id,profile_id,profile_type,school_id,publisher_id,time_to_study,average_rating,action_type,action_info,action_start,action_end,action_user_id,action_profile_id,action_profile_type,action_school_id
156,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,загрузка в библиотеку,NaN,2014-08-30 20:45:22.828479,2014-08-30 20:45:22.828479,423755.0,444837.0,PublisherProfile,NaN
41562,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,обновлён,NaN,2015-12-03 21:18:31.35267,2015-12-03 21:18:31.35267,423755.0,444837.0,PublisherProfile,NaN
113222,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-04-28 07:10:38.347353,2016-04-28 07:10:38.347353,229154.0,244998.0,TeacherProfile,25.0
114271,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-04-29 05:26:52.746096,2016-04-29 05:26:52.746096,78343.0,79454.0,TeacherProfile,19.0
179681,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-05-31 13:16:31.357204,2016-05-31 13:16:31.357204,78069.0,79115.0,TeacherProfile,4.0
186759,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-06-20 06:15:46.008755,2016-06-20 06:15:46.008755,77043.0,77842.0,TeacherProfile,17.0
214237,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-08-24 10:16:13.076055,2016-08-24 10:16:13.076055,2688054.0,2792272.0,TeacherProfile,905.0
214546,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-08-24 18:40:53.255401,2016-08-24 18:40:53.255401,3033115.0,3144841.0,TeacherProfile,1137.0
214593,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-08-24 20:00:21.90159,2016-08-24 20:00:21.90159,2518392.0,2609442.0,TeacherProfile,791.0
240911,Book,2,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,1.0,NaN,5.0,добавление в избранное,NaN,2016-10-16 15:29:40.792244,2016-10-16 15:29:40.792244,2433476.0,2517917.0,TeacherProfile,85.0


In [23]:
pd.crosstab(df.action_type, df.material_type)

material_type,AtomicObject,Book,ComposedDocument,DigitalDocument,GameApp,LessonTemplate,TestSpecification
action_type,,,,,,,
добавление в избранное,193708,69132,8482,26,74594,280885,29412
загрузка в библиотеку,379893,1497,614,5,13908,25587,14297
запуск,0,0,0,0,597959,1323796,0
использование в ЭУП,4510,0,0,0,0,0,0
использование в сценарии урока,56375,0,0,0,0,0,0
копирование,0,0,5942,0,0,360705,0
обновлён,379893,1497,614,5,13908,25587,14297
одобрено модерацией,405680,881,793,7,15890,45661,17185
отправка на модерацию,218061,416,966,7,14883,62128,18798


In [27]:
df[(df.material_id == 2000) & (df.material_type == 'Book')]

,material_type,material_id,content_type,logical_type,book_type,game_app_category,game_app_version_id,game_app_version,user_id,profile_id,profile_type,school_id,publisher_id,time_to_study,average_rating,action_type,action_info,action_start,action_end,action_user_id,action_profile_id,action_profile_type,action_school_id
471,Book,2000,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,15.0,NaN,0.0,загрузка в библиотеку,NaN,2015-05-19 04:51:43.106893,2015-05-19 04:51:43.106893,423755.0,444837.0,PublisherProfile,NaN
41745,Book,2000,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,15.0,NaN,0.0,обновлён,NaN,2015-12-03 21:18:40.157404,2015-12-03 21:18:40.157404,423755.0,444837.0,PublisherProfile,NaN
44312,Book,2000,NaN,NaN,Учебник,NaN,NaN,NaN,423755.0,444837.0,PublisherProfile,NaN,15.0,NaN,0.0,удаление,NaN,2015-12-15 08:23:04.140039,2015-12-15 08:23:04.140039,423755.0,444837.0,PublisherProfile,NaN


In [29]:
len(df.action_profile_id.unique())

187970

In [31]:
df.action_profile_id.unique().shape

(187970,)

In [32]:
df.action_user_id.unique().shape

(179798,)

In [36]:
df.groupby(['action_profile_id', 'action_user_id']
          ).action_type.count().reset_index().action_profile_id.value_counts().max()

1

In [38]:
df.groupby(['action_profile_id', 'action_user_id']
          ).action_type.count().reset_index().action_user_id.value_counts()

425382.0      13
457230.0       7
133822.0       7
2285201.0      7
2285257.0      7
4303188.0      6
510903.0       5
7777777.0      5
2278247.0      5
2278219.0      5
2365051.0      5
2277480.0      5
530839.0       5
61.0           4
5609724.0      4
2585043.0      4
136057.0       4
504188.0       4
4262978.0      4
2345710.0      4
3991055.0      4
425562.0       4
3279771.0      4
4148867.0      4
3669499.0      4
448198.0       4
5067094.0      4
623390.0       4
79231.0        4
599476.0       4
3664223.0      4
3671563.0      4
2606358.0      4
3663475.0      4
4327506.0      4
536433.0       4
509414.0       4
487194.0       4
4287955.0      4
2354640.0      4
2690276.0      4
2348709.0      4
531200.0       4
504232.0       4
460495.0       4
3079527.0      4
2448879.0      4
2278191.0      4
2346575.0      4
12382188.0     4
590523.0       4
3376172.0      4
544587.0       4
3663268.0      4
3392411.0      4
541972.0       4
3282259.0      4
544449.0       4
2618428.0     

In [39]:
df[df.action_user_id == 425382]

,material_type,material_id,content_type,logical_type,book_type,game_app_category,game_app_version_id,game_app_version,user_id,profile_id,profile_type,school_id,publisher_id,time_to_study,average_rating,action_type,action_info,action_start,action_end,action_user_id,action_profile_id,action_profile_type,action_school_id
11050,AtomicObject,7700,Изображение,Фотография,NaN,NaN,NaN,NaN,425596.0,441289.0,TeacherProfile,1135.0,NaN,NaN,3.0,просмотр,NaN,2015-10-02 13:21:36.279767,2015-10-02 13:21:36.279767,425382.0,2356122.0,ContentManagerProfile,NaN
21193,AtomicObject,10696,Изображение,Фотография,NaN,NaN,NaN,NaN,449914.0,472336.0,AuthorProfile,465.0,NaN,NaN,4.0,просмотр,NaN,2015-10-26 10:11:49.035507,2015-10-26 10:11:49.035507,425382.0,2356122.0,ContentManagerProfile,NaN
21194,AtomicObject,10714,Изображение,Фотография,NaN,NaN,NaN,NaN,449914.0,472336.0,AuthorProfile,465.0,NaN,NaN,3.7,просмотр,NaN,2015-10-26 10:13:32.251686,2015-10-26 10:13:32.251686,425382.0,2356122.0,ContentManagerProfile,NaN
21256,AtomicObject,9732,Текст,Правило,NaN,NaN,NaN,NaN,441441.0,463653.0,AuthorProfile,395.0,NaN,NaN,3.0,просмотр,NaN,2015-10-26 13:00:14.917653,2015-10-26 13:00:14.917653,425382.0,2356122.0,ContentManagerProfile,NaN
21629,AtomicObject,10715,Текст,Событие (факт),NaN,NaN,NaN,NaN,449914.0,472336.0,AuthorProfile,465.0,NaN,NaN,3.0,просмотр,NaN,2015-10-27 07:35:09.937658,2015-10-27 07:35:09.937658,425382.0,2356122.0,ContentManagerProfile,NaN
21682,AtomicObject,10972,Текст,Правило,NaN,NaN,NaN,NaN,462262.0,485196.0,AuthorProfile,134.0,NaN,NaN,4.4,просмотр,NaN,2015-10-27 09:07:50.762866,2015-10-27 09:07:50.762866,425382.0,2356122.0,ContentManagerProfile,NaN
25353,AtomicObject,12039,Текст,Задание,NaN,NaN,NaN,NaN,443143.0,3399603.0,AuthorProfile,572.0,NaN,NaN,1.0,просмотр,NaN,2015-11-03 08:15:10.475873,2015-11-03 08:15:10.475873,425382.0,2356122.0,ContentManagerProfile,NaN
38275,AtomicObject,15216,Видео,NaN,NaN,NaN,NaN,NaN,443174.0,465425.0,AuthorProfile,659.0,NaN,NaN,4.5,просмотр,NaN,2015-11-23 20:22:10.660413,2015-11-23 20:22:10.660413,425382.0,2356122.0,ContentManagerProfile,NaN
39957,Book,2114,NaN,NaN,UGC,NaN,NaN,NaN,5214352.0,5382977.0,TeacherProfile,1043.0,NaN,NaN,0.0,просмотр,NaN,2015-11-26 21:37:23.139648,2015-11-26 21:37:23.139648,425382.0,2356122.0,ContentManagerProfile,NaN
40224,Book,2532,NaN,NaN,UGC,NaN,NaN,NaN,425564.0,442273.0,AuthorProfile,1135.0,NaN,NaN,0.0,просмотр,NaN,2015-11-27 13:48:26.656102,2015-11-27 13:48:26.656102,425382.0,2356122.0,ContentManagerProfile,NaN


In [40]:
df.action_profile_type.value_counts()

TeacherProfile              8882400
MethodologistProfile         537431
StudentProfile               364104
ParentProfile                348950
AuthorProfile                125511
PublisherProfile              50160
AdminProfile                  18900
ContentManagerProfile           420
PrincipalProfile                400
DeputyProfile                   250
MaterialModeratorProfile         76
MaterialViewerProfile            72
SchoolAdminProfile               27
JuryProfile                      19
StaffProfile                     17
CuratorProfile                   13
AdminSecurityProfile              1
Name: action_profile_type, dtype: int64

In [42]:
pd.crosstab(df.action_type, df.action_profile_type).T

action_type,добавление в избранное,загрузка в библиотеку,запуск,использование в ЭУП,использование в сценарии урока,копирование,обновлён,одобрено модерацией,отправка на модерацию,оценка,просмотр,удаление
action_profile_type,,,,,,,,,,,,
AdminProfile,0,0,18900,0,0,0,0,0,0,0,0,0
AdminSecurityProfile,0,0,1,0,0,0,0,0,0,0,0,0
AuthorProfile,640,11972,10229,622,777,559,11972,5,8183,43997,24583,11972
ContentManagerProfile,0,0,0,0,0,0,0,296,12,0,112,0
CuratorProfile,0,0,1,0,0,0,0,0,0,0,12,0
DeputyProfile,0,0,0,0,0,0,0,0,0,0,250,0
JuryProfile,0,0,0,0,0,0,0,0,0,9,10,0
MaterialModeratorProfile,0,0,0,0,0,0,0,26,14,1,35,0
MaterialViewerProfile,3,0,3,0,0,0,0,0,0,3,63,0


In [74]:
del_list = ['time_to_study', 'action_school_id', 'school_id', 'publisher_id', 'action_profile_id', 
            'profile_id', 'profile_type', 'action_profile_type']

In [63]:
df = df[df.action_start.isnull() != True]

In [64]:
df.action_start = df.action_start.apply(lambda x: datetime.strptime(x[:19], "%Y-%m-%d %H:%M:%S"))

In [65]:
df.action_end = df.action_end.apply(lambda x: datetime.strptime(x[:19], "%Y-%m-%d %H:%M:%S"))

In [66]:
df.head(2)

,material_type,material_id,content_type,logical_type,book_type,game_app_category,game_app_version_id,game_app_version,user_id,profile_id,profile_type,school_id,publisher_id,time_to_study,average_rating,action_type,action_info,action_start,action_end,action_user_id,action_profile_id,action_profile_type,action_school_id
0,Book,24853,NaN,NaN,ХудЛит,NaN,NaN,NaN,13057293.0,13340607.0,PublisherProfile,NaN,383.0,NaN,0.0,загрузка в библиотеку,NaN,2007-06-24 12:26:10,2007-06-24 12:26:10,13057293.0,13340607.0,PublisherProfile,NaN
1,Book,24724,NaN,NaN,ХудЛит,NaN,NaN,NaN,13057293.0,13340607.0,PublisherProfile,NaN,383.0,NaN,0.0,загрузка в библиотеку,NaN,2007-09-25 21:40:27,2007-09-25 21:40:27,13057293.0,13340607.0,PublisherProfile,NaN


In [69]:
df['duration'] = df.action_end - df.action_start

In [70]:
df.duration.value_counts()

0 days 00:00:00      8235371
0 days 00:20:00        94668
0 days 00:00:30        42007
0 days 00:00:31        28858
0 days 00:01:00        20103
0 days 00:01:01        17006
0 days 00:00:05        11938
0 days 00:01:30        11600
0 days 00:00:04        11466
0 days 00:00:06        10696
0 days 00:01:31        10662
0 days 00:00:07         9702
0 days 00:00:08         9162
0 days 00:00:09         8538
0 days 00:00:03         8481
0 days 00:00:10         8179
0 days 00:00:12         7817
0 days 00:02:01         7684
0 days 00:00:13         7654
0 days 00:01:02         7529
0 days 00:00:11         7390
0 days 00:00:32         7135
0 days 00:02:00         6912
0 days 00:00:15         6796
0 days 00:00:16         6791
0 days 00:00:14         6399
0 days 00:02:31         5881
0 days 00:00:17         5703
0 days 00:00:19         5673
0 days 00:00:20         5552
0 days 00:00:18         5518
0 days 00:00:21         5223
0 days 00:00:33         5116
0 days 00:00:22         5107
0 days 00:00:0

In [72]:
df['is_zero_duration'] = df.action_end == df.action_start

In [73]:
pd.crosstab(df.action_type, df.is_zero_duration)

is_zero_duration,False,True
action_type,,
добавление в избранное,0,656239
загрузка в библиотеку,0,435801
запуск,1674054,247701
использование в ЭУП,0,4510
использование в сценарии урока,0,56375
копирование,0,366647
обновлён,0,435801
одобрено модерацией,0,486097
отправка на модерацию,0,315259


In [75]:
del_list += ['is_zero_duration', 'duration', 'action_end']

In [76]:
df.drop(del_list, axis = 1, inplace = True)

In [78]:
df.drop(['game_app_version_id', 'game_app_version'], axis = 1, inplace = True)

In [79]:
df.head()

,material_type,material_id,content_type,logical_type,book_type,game_app_category,user_id,average_rating,action_type,action_info,action_start,action_user_id
0,Book,24853,NaN,NaN,ХудЛит,NaN,13057293.0,0.0,загрузка в библиотеку,NaN,2007-06-24 12:26:10,13057293.0
1,Book,24724,NaN,NaN,ХудЛит,NaN,13057293.0,0.0,загрузка в библиотеку,NaN,2007-09-25 21:40:27,13057293.0
2,Book,24855,NaN,NaN,ХудЛит,NaN,13057293.0,0.0,загрузка в библиотеку,NaN,2007-10-12 02:05:57,13057293.0
3,Book,24860,NaN,NaN,ХудЛит,NaN,13057293.0,0.0,загрузка в библиотеку,NaN,2007-10-13 08:34:49,13057293.0
4,Book,24830,NaN,NaN,ХудЛит,NaN,13057293.0,0.0,загрузка в библиотеку,NaN,2007-12-18 17:26:05,13057293.0


In [80]:
explore_list = ['logical_type', 'book_type', 'game_app_category', 'action_info']

### Задачи:
1. Посмотреть области значений оставшихся 4 полей.
2. Посмотреть содержимое остальных 6 файлов (первичный анализ).